# LangChain Embedding with VectorDB chromadb

## 安装 python 依赖

In [ ]:

# 通过pip安装
! pip install langchain
! pip install openai
# 向量数据库
! pip install chromadb
# 中文分词
! pip install jieba
# 处理非结构化数据
! pip install unstructured
! pip install cryptography
! pip install paramiko

## 导入依赖

In [1]:
import os
import jieba as jb
from langchain.llms import OpenAI

from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain, ChatVectorDBChain
from langchain.chat_models import ChatOpenAI

## 准备数据

将下面的数据保存为文件： 2022世界杯.txt

在卡塔尔当地时间2022年12月18日，梅西带领阿根廷男足在2022年世界杯决赛中通过点球大战以7-5击败法国男足，为阿根廷队夺得队史的第三座世界杯冠军奖杯，这也是阿根廷队时隔36年再度捧起世界杯冠军奖杯。
夺冠后梅西深情亲吻大力神杯，尽情宣泄心中的激动和喜悦。值得一提的是，阿根廷夺得2022世界杯冠军之后，梅西等阿根廷球员纷纷在个人社交媒体发表了对于夺冠的喜悦之情。
对于梅西而言，个人足球职业生涯再无遗憾，而对于年轻的阿根廷小将们则意味着职业生涯奠定了一个非常好的开端。而对于梅西的老帮手迪马利亚来说，他改变了自己从阿根廷国家队退役的决定，希望能够以冠军身份继续为潘帕斯雄鹰效力。
阿根廷队本来是准备在首都布宜诺斯艾利斯的市中心共和国广场进行庆祝，但是由于球迷实在过于热情，最终庆祝仪式被迫提前结束，球员们改为直升飞机绕城飞行庆祝，但这也依然不能熄灭球迷的火热激情。所有看到阿根廷夺冠球员的球迷都激动万分，有的球迷更是兴奋的直接泣不成声，场面十分令人动容。
梅西在夺得2022世界杯冠军之后也是情绪激动，以世界杯冠军结束自己的世界杯旅程，对于梅西而言毫无疑问是最完美的结局，而梅西的家人们也在比赛现场见证了梅西的巅峰时刻，一家人的幸福合影令人羡慕。


## 开始读取本地训练数据并分词处理

本地的 txt 文件内是中文，需要使用 jieba 对中文进行分词处理

In [2]:
def split_text():
    file='2022世界杯.txt'
    my_file = f"./tmp/data/{file}"

    if not os.path.exists("./tmp/data/cut"):
        os.makedirs("./tmp/data/cut")

    # 从本地读取文件
    with open(my_file, "r", encoding='utf-8') as f:
        data = f.read()
        # 对内容按空格进行分词处理
        cut_data = " ".join(w for w in list(jb.cut(data)))
        # 保存分词后的内容
        cut_file = f"./tmp/data/cut/cut_{file}"
        with open(cut_file, 'w') as f:
            f.write(cut_data)
            f.close()
    print(cut_data)
    print("写入文件完成")

In [3]:
split_text()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Dell\AppData\Local\Temp\jieba.cache
Loading model cost 0.609 seconds.
Prefix dict has been built successfully.


在 卡塔尔 当地 时间 2022 年 12 月 18 日 ， 梅西 带领 阿根廷 男足 在 2022 年 世界杯 决赛 中 通过 点球 大战 以 7 - 5 击败 法国 男足 ， 为 阿根廷队 夺得 队史 的 第三座 世界杯 冠军 奖杯 ， 这 也 是 阿根廷队 时隔 36 年 再度 捧起 世界杯 冠军 奖杯 。 
 夺冠 后梅西 深情 亲吻 大力神杯 ， 尽情 宣泄 心中 的 激动 和 喜悦 。 值得一提的是 ， 阿根廷 夺得 2022 世界杯 冠军 之后 ， 梅西 等 阿根廷 球员 纷纷 在 个人 社交 媒体 发表 了 对于 夺冠 的 喜悦之情 。 
 对于 梅西 而言 ， 个人 足球 职业生涯 再 无 遗憾 ， 而 对于 年轻 的 阿根廷 小将 们 则 意味着 职业生涯 奠定 了 一个 非常 好 的 开端 。 而 对于 梅西 的 老 帮手 迪 马利亚 来说 ， 他 改变 了 自己 从 阿根廷 国家队 退役 的 决定 ， 希望 能够 以 冠军 身份 继续 为 潘帕斯 雄鹰 效力 。 
 阿根廷队 本来 是 准备 在 首都 布宜诺斯艾利斯 的 市中心 共和国 广场 进行 庆祝 ， 但是 由于 球迷 实在 过于 热情 ， 最终 庆祝 仪式 被迫 提前结束 ， 球员 们 改为 直升飞机 绕城 飞行 庆祝 ， 但 这 也 依然 不能 熄灭 球迷 的 火热 激情 。 所有 看到 阿根廷 夺冠 球员 的 球迷 都 激动 万分 ， 有 的 球迷 更是 兴奋 的 直接 泣不成声 ， 场面 十分 令人 动容 。 
 梅西 在 夺得 2022 世界杯 冠军 之后 也 是 情绪 激动 ， 以 世界杯 冠军 结束 自己 的 世界杯 旅程 ， 对于 梅西 而言 毫无疑问 是 最 完美 的 结局 ， 而梅西 的 家 人们 也 在 比赛 现场 见证 了 梅西 的 巅峰 时刻 ， 一家人 的 幸福 合影 令人羡慕 。
写入文件完成


## 通过openai的Embeddings训练

    DirectoryLoader 是由 langchain 提供的目录加载器。langchain 提供了很多种不同文档加载器，根据资源来源不同，使用对应的加载器。

    TokenTextSplitter 是由 langchain 提供的文本分割器，它也提供一系列其它的 splitter 。这里需要自行设定要分块的大小

In [4]:
# 通过目录加载器读取分词数据
loader = DirectoryLoader('./tmp/data/cut', glob='**/*.txt')
docs = loader.load()
# 这里要进行分块处理，因为openai的接口是有限制的,chunk_size表示块大小，chunk_overlap表示块重叠部分多少
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
doc_texts = text_splitter.split_documents(docs)
print(doc_texts)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


[Document(page_content='在 卡塔尔 当地 时间 2022 年 12 月 18 日 ， 梅西 带领 阿根廷 男足 在 2022 年 世界杯 决赛 中 通过 点球 大战 以 7 - 5 击败 法国 男足 ， 为 阿根廷队 夺得 队史 的 第三座 世界杯 冠军 奖杯 ， 这 也 是 阿根廷队 时隔 36 年 再度 捧起 世界杯 冠军 奖杯 。 夺冠 后梅西 深情 亲吻 大力神杯 ， 尽情 宣泄 心中 的 激动 和 喜悦 。 值得一提的是 ， 阿根廷 夺得 2022 世界杯 冠军 之后 ， 梅西 等 阿根廷 球员 纷纷 在 个人 社交 媒体 发表 了 对于 夺冠 的 喜悦之情 。 对于 梅西 而言 ， 个人 足球 职业生涯 再 无 遗憾 ， 而 对于 年轻 的 阿根廷 小将 们 则 意味着 职业生涯 奠定 了 一个 非常 好 的 开端 。 而 对于 梅西 的 老 帮手 迪 马利亚 来说 ， 他 改变 了 自己 从 阿根廷 国家队 退役 的 决定 ， 希望 能够 以 冠军 身份 继续 为 潘帕斯 雄鹰 效力 。 阿根廷队 本来 是 准备 在 首都 布宜诺斯艾利斯 的 市中心 共和国 广场 进行 庆祝 ， 但是 由于 球迷 实在 过于 热情 ， 最终 庆祝 仪式 被迫 提前结束 ， 球员 们 改为 直升飞机 绕城 飞行 庆祝 ， 但 这 也 依然 不能 熄灭 球迷 的 火热 激情 。 所有 看到 阿根廷 夺冠 球员 的 球迷 都 激动 万分 ， 有 的 球迷 �', metadata={'source': 'data\\cut\\cut_2022世界杯.txt'}), Document(page_content='��是 兴奋 的 直接 泣不成声 ， 场面 十分 令人 动容 。 梅西 在 夺得 2022 世界杯 冠军 之后 也 是 情绪 激动 ， 以 世界杯 冠军 结束 自己 的 世界杯 旅程 ， 对于 梅西 而言 毫无疑问 是 最 完美 的 结局 ， 而梅西 的 家 人们 也 在 比赛 现场 见证 了 梅西 的 巅峰 时刻 ， 一家人 的 幸福 合影 令人羡慕 。', metadata={'source': 'data\\cut\\cut_2022世界杯.txt'})]



## 通过调用 openai 的 embeddings 接口，去训练数据，训练以后，接口会返回一组向量数据。

langchain 也提供封装了很多种向量数据库。网上很多是用 Pinecone 来保存，这个是一个三方的服务商来着，要申请。
这里通过Chroma保存向量数据。Chroma 是一款开源的向量数据库，可以来替换 Pinecone。通过 chroma 把向量数据储存在本地目录

In [5]:

# 调用openai的embeddings接口，然后保存返回的向量数据
os.environ["OPENAI_API_KEY"] = "sk-"
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
# 指定保存目录
vectordb = Chroma.from_documents(doc_texts, embeddings, persist_directory="./tmp/data/cut")
vectordb.persist()

Using embedded DuckDB with persistence: data will be stored in: ./data/cut


它会在./tmp/data/cut目录下生成

chroma-collections.parquet文件

chroma-embeddings.parquet文件

./tmp/data/cut/index/目录下生成相关索引

到此，数据模型训练完毕。

## 接入聊天api

模型训练完了，现在可以再次向chatgpt发起提问2022年世界杯冠军

In [6]:

## 创建聊天对象
chain = ChatVectorDBChain.from_llm(OpenAI(temperature=0, 
          model_name="gpt-3.5-turbo"), vectordb,
                 return_source_documents=True)
          
# 聊天对话，获取回复
def get_answer(question):
    chat_history=[]
    result=chain({
        "question":question,
        "chat_history":chat_history
    })
    return result["answer"]

question = '2022年世界杯冠军是谁？'
print(get_answer(question))

c:\Users\Dell\.conda\envs\py39\lib\site-packages\langchain\llms\openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Dell\.conda\envs\py39\lib\site-packages\langchain\llms\openai.py:687: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Dell\.conda\envs\py39\lib\site-packages\langchain\chains\conversational_retrieval\base.py:233: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(
Chroma collection langchain contains fewer than 4 elements.
Chroma collection langchain contains fewer than 3 elements.


阿根廷队。


In [7]:
question = '谁带阿根廷男足赢了2022年世界杯？'
print(get_answer(question))

Chroma collection langchain contains fewer than 4 elements.
Chroma collection langchain contains fewer than 3 elements.


梅西带领阿根廷男足赢了2022年世界杯。


输出

question = '2022年世界杯冠军是谁？'

answer = 阿根廷男足。

question = ‘谁带阿根廷男足赢了2022年世界杯？’

answer = 梅西带领阿根廷男足赢得了2022年世界杯。